In [16]:
import folium
import pandas as pd

In [2]:
df = pd.read_csv("../data/usa-hospital-beds_dataset_usa-hospital-beds.csv")
df.head()

,X,Y,OBJECTID,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_ADDRESS,HQ_ADDRESS1,HQ_CITY,HQ_STATE,HQ_ZIP_CODE,...,CNTY_FIPS,FIPS,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE
0,-112.066157,33.495498,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,NaN,Phoenix,AZ,85012,...,13.0,4013.0,62.0,62.0,0,0,NaN,NaN,0,0.0
1,-110.965885,32.181263,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,NaN,Tucson,AZ,85723,...,19.0,4019.0,295.0,295.0,2,2,NaN,NaN,0,2.0
2,-119.779742,36.773323,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,NaN,Fresno,CA,93703,...,19.0,6019.0,54.0,54.0,2,2,NaN,NaN,0,2.0
3,-72.957610,41.284400,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,NaN,West Haven,CT,6516,...,9.0,9009.0,216.0,216.0,1,1,NaN,NaN,0,2.0
4,-75.606533,39.740206,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,NaN,Wilmington,DE,19805,...,3.0,10003.0,62.0,62.0,0,0,NaN,NaN,0,1.0


In [3]:
# Make a dataframe with only the features to display in the map
# Choosing features
features = ["Y","X","BED_UTILIZATION","HOSPITAL_NAME","HQ_ADDRESS","HQ_CITY","STATE_NAME","HQ_ZIP_CODE"]

# make the sub-dataframe
sub_df = df[features]

#clean_sub_df.head()

In [49]:
def map_state(state_name):
    
    # Filter dataframe by state
    is_state = sub_df["STATE_NAME"] == state_name
    
    # Make chosen state dataframe
    state_df = sub_df[is_state].reset_index(drop=True)    
    
    # Coordinates are read [Y, X] AKA [latitute, longitude]   
    center_coords = [state_df["Y"].mean(),state_df["X"].mean()]
    
    my_map = folium.Map(location = center_coords, zoom_start = 7.5)   
    
    for i in range(len(state_df)):    
        
        # Make a string of what will be added to the folium marker                
        popup_str1 = "<b>" + state_df.loc[i, "HOSPITAL_NAME"] + "</b><br>" 
        popup_str2 = state_df.loc[i, "HQ_ADDRESS"] + ", " + state_df.loc[i, "HQ_CITY"] + ", "
        popup_str3 = str(state_df.loc[i, "HQ_ZIP_CODE"]) + "<br>" + state_df.loc[i, "STATE_NAME"]         
        
        popup_string = popup_str1 + popup_str2 + popup_str3
                
        pp = folium.Html(popup_string, script=True)
        popup = folium.Popup(pp, max_width=2560)       
        
        tooltip_string = "Beds Available: "
        
        # Iterate through dataframe and round the bed capacity for display
        cap = state_df.loc[i, "BED_UTILIZATION"]
        
        if int(math.isnan(cap)):
            tooltip_string += "Data Not Available" 
        else:           
            # Use an f-string to calculate bed availability percentage and round the result in a single line
            #tooltip_string += f'{1-cap:.2f}' +"%"                        
            tooltip_string += f'{100*(1-cap):.2f}' +"%"                        
                
        # Obtain marker coordinates from dataframe
        marker_coords = [state_df.loc[i, "Y"], state_df.loc[i, "X"]]
        
        # Add everything to the markers and add the markers to the map
        folium.Marker(marker_coords,popup=popup, tooltip=tooltip_string).add_to(my_map)
    
    return my_map    

In [50]:
map_state("California")

In [85]:
my_map.save("../map.html")